## Problem 4(a)

In [72]:
# Import packages
import numpy as np
import pandas as pd
import scipy.stats as sts
import os
import matplotlib.pyplot as plt
import scipy.optimize as opt
import warnings
warnings.filterwarnings('ignore')
from scipy import special
from scipy.integrate import quad

In [73]:
# Load data
data = np.loadtxt('data/MacroSeries.txt', delimiter=",")
c = data[:, 0]
k = data[:, 1]
w = data[:, 2]
r = data[:, 3]

In [74]:
def get_z(r, k, alpha):
    
    z = np.log(r) - np.log(alpha) - (alpha - 1) * np.log(k)
    
    return z

def data_moments(c, k, w, r, alpha, beta, rho, mu):
    
    z = get_z(r, k, alpha)
    A = z[1:] - rho * z[:-1] - (1 - rho) * mu
    B = (z[1:] - rho * z[:-1] - (1 - rho) * mu) * z[:-1]
    C = beta * alpha * np.exp(z[1:]) * k[1:] ** (alpha - 1) * c[:-1]/c[1:] - 1
    D = (beta * alpha * np.exp(z[1:]) * k[1:] ** (alpha - 1) * c[:-1]/c[1:] - 1) * w[:-1]
    
    return A.mean(), B.mean(), C.mean(), D.mean()

def err_vec(c, k, w, r, alpha, beta, rho, mu, simple = True):
    mo_moments = np.zeros(4)
    da_moments = np.array(data_moments(c, k, w, r, alpha, beta, rho, mu)).reshape(1, 4)
    if simple:
        err_vec = (mo_moments - da_moments)[0]
    else:
        err_vec = (mo_moments - da_moments) / da_moments
    return err_vec

def crit(params, *args):
    alpha, beta, rho, mu = params
    c, k, w, r, weight  = args
    err = err_vec(c, k, w, r, alpha, beta, rho, mu)
    crit_val = np.dot(np.dot(err.T, weight), err) 
    
    return crit_val

In [75]:
weight = np.identity(4)
alpha_init = 0.42
beta_init = 0.9
rho_init = 0.74
mu_init = 8

params_init = (alpha_init, beta_init, rho_init, mu_init)
bnds = ((1e-5, 1-1e-5), (1e-5, 1-1e-5), (-1+1e-5, 1-1e-5), (1e-5, None))
args = (c, k, w, r, weight)
results = opt.minimize(crit, params_init, args=(args), bounds = bnds, method = 'L-BFGS-B')
print(results)

      fun: 0.0027665192987751212
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -2.73431460e-03,   1.48191769e+02,   1.88093898e-05,
        -1.11645992e-03])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 275
      nit: 35
   status: 0
  success: True
        x: array([ 0.53416437,  0.99      ,  0.78108453,  8.00523255])


It seems that the values of $\alpha, \rho$, and especially $\mu$ change a lot based on initial guesses.